In [1]:
from UAV import *
import numpy as np

### EVD algorithm

In [2]:
def EVD(DM, final_dimension):
    """
    Return the relative map of the nodes in the network
    """
    n = len(DM)
    # Centering Matrix definition
    H = np.eye(n) - np.ones((n,n))/n

    # Double centered matrix
    B = -1/2*H@DM@H

    # Eiegenvalue decomposition
    ev, EV = np.linalg.eig(B)

    # Sort according to the eigenvalues magnitude
    seq = np.argsort(ev)[::-1]

    LAMBDA = np.diag(ev[seq][:final_dimension])
    U = EV[:,seq][:, :final_dimension]

    # Relative map
    X1 = np.sqrt(LAMBDA)@U.T

    return X1.real

### Generate random coordinates

In [3]:
anchor = np.random.uniform(size=[3,1], low=-5, high=5)
anchor = anchor + np.hstack([np.zeros([3,1]), np.random.uniform(low=-1,high=1, size=[3,3])])

nodes = np.random.uniform(size=[3,6], low=-5, high=5)

# Define the coordinates matrix
X = np.hstack([anchor, nodes])
print(X)

N = X.shape[1]
e = np.ones(N).reshape(-1,1)

[[ 4.0651259   3.60516298  4.55418851  3.79158146 -0.19268371  3.7537264
   0.46520449 -4.90653102 -4.12930533 -2.72559897]
 [-3.32439712 -4.1087798  -4.26848511 -3.13758634  3.17023996 -4.02727448
   3.71318793 -1.91112198  2.82744067 -4.81848186]
 [-1.51422265 -1.1409655  -0.58062112 -2.343711    1.06013458 -1.06374833
   2.33012582  1.30089469 -3.75718678 -0.91486624]]


### Apply the EVD algorithm and get the relative map S

In [4]:
# Compute the associated distance matrix
Phi = np.diag(X.T @ X).reshape(-1,1)
D = Phi @ e.T - 2* X.T @ X + e @ Phi.T

S =  EVD(D,3)

### Solve the ambiguity

In [5]:
# Define P and Q, two sets of corresponding nodes
# P = anchors from the relative map
# Q = true position of the anchors

P = S[:,:4]
Q = X[:,:4]

print(P)
print(Q)

[[ 3.65606239e+00  3.77999484e+00  4.52408105e+00  3.43145409e+00]
 [ 7.32219835e-01 -6.68969227e-02  5.57253510e-01  5.24080026e-01]
 [ 5.60629074e-01  1.88392736e-03 -5.43013344e-01  1.39967894e+00]]
[[ 4.0651259   3.60516298  4.55418851  3.79158146]
 [-3.32439712 -4.1087798  -4.26848511 -3.13758634]
 [-1.51422265 -1.1409655  -0.58062112 -2.343711  ]]


### Notes

Apply the following equation
\begin{equation}
(R,t) = \argmin_{R,t} \sum_{i=1}^N ||(R\,p_i + t) - q_i||^2
\end{equation}

def toVector(w, z):
    assert w.shape == (2, 4)
    assert z.shape == (2, 4)
    return np.hstack([w.flatten(), z.flatten()])

def toWZ(vec):
    assert vec.shape == (2*2*4,)
    return vec[:2*4].reshape(2,4), vec[2*4:].reshape(2,4)

def doOptimization(f_of_w_z, w0, z0):
    def f(x): 
        w, z = toWZ(x)
        return f_of_w_z(w, z)

    result = minimize(f, toVec(w0, z0))
    # Different optimize functions return their
    # vector result differently. In this case it's result.x:
    result.x = toWZ(result.x) 
    return result

### Solving the problem via the SVD approach

In [10]:
#1. Compute the weighted centroids of both point sets
P_bar, Q_bar = 1/4*np.sum(P, axis=1).reshape(-1,1), 1/4*np.sum(Q, axis=1).reshape(-1,1)

#2. Compute the centered vectors
P1, Q1 = P - P_bar, Q - Q_bar

#3. Compute the 3 × 3 covariance matrix
H = P1 @ Q1.T

#4. Compute the singular value decomposition
SS, VV, DD = np.linalg.svd(H)
R = DD @ SS.T

#5. Compute the optimal translation as
t = Q_bar - R @ P_bar

def f(x, i=3):
    return(x[:,:i])

X_hat = R@S + t

print(X)
print()
print()

print(X_hat)
print()
print()

print(X-X_hat)
print(np.max(X-X_hat))

[[ 4.0651259   3.60516298  4.55418851  3.79158146 -0.19268371  3.7537264
   0.46520449 -4.90653102 -4.12930533 -2.72559897]
 [-3.32439712 -4.1087798  -4.26848511 -3.13758634  3.17023996 -4.02727448
   3.71318793 -1.91112198  2.82744067 -4.81848186]
 [-1.51422265 -1.1409655  -0.58062112 -2.343711    1.06013458 -1.06374833
   2.33012582  1.30089469 -3.75718678 -0.91486624]]


[[  3.93479011   4.40522411   4.08330031   3.59274432   9.68588189
    4.36313207  10.15843217  12.49322336   9.17619222   8.99681827]
 [ -3.87108644  -3.65897262  -2.61303578  -4.69615353  -6.59099834
   -3.53415796  -5.42194372  -7.8747617  -12.43331475  -7.25252772]
 [ -1.02647747  -1.8630434   -1.65683409  -1.0331653    4.11780005
   -1.73141414   4.984111    -2.09262237   2.01336624  -4.43927506]]


[[  0.13033579  -0.80006114   0.4708882    0.19883714  -9.8785656
   -0.60940566  -9.69322768 -17.39975439 -13.30549755 -11.72241723]
 [  0.54668932  -0.44980719  -1.65544933   1.55856719   9.76123829
   -0.49311652

In [7]:
def RMSE(S, S_hat):
    f = 0

    for i in range(S.shape[1]):
        tmp = 0
        for j in range(S.shape[0]):
            print(" %.3f | %.3f | %.3f" % (S[j,i], S_hat[j,i], S[j,i] - S_hat[j,i]))
            tmp += (S[j,i] - S_hat[j,i])**2
        f += np.sqrt(tmp)
    
    return f/S.shape[1]

anchors_real = X[:,:4]
anchors_estim = X_hat[:,:4]

#print(anchors_real - anchors_estim)
print(anchors_real)
RMSE(X, X_hat)

[[ 4.0651259   3.60516298  4.55418851  3.79158146]
 [-3.32439712 -4.1087798  -4.26848511 -3.13758634]
 [-1.51422265 -1.1409655  -0.58062112 -2.343711  ]]
 4.065 | 3.935 | 0.130
 -3.324 | -3.871 | 0.547
 -1.514 | -1.026 | -0.488
 3.605 | 4.405 | -0.800
 -4.109 | -3.659 | -0.450
 -1.141 | -1.863 | 0.722
 4.554 | 4.083 | 0.471
 -4.268 | -2.613 | -1.655
 -0.581 | -1.657 | 1.076
 3.792 | 3.593 | 0.199
 -3.138 | -4.696 | 1.559
 -2.344 | -1.033 | -1.311
 -0.193 | 9.686 | -9.879
 3.170 | -6.591 | 9.761
 1.060 | 4.118 | -3.058
 3.754 | 4.363 | -0.609
 -4.027 | -3.534 | -0.493
 -1.064 | -1.731 | 0.668
 0.465 | 10.158 | -9.693
 3.713 | -5.422 | 9.135
 2.330 | 4.984 | -2.654
 -4.907 | 12.493 | -17.400
 -1.911 | -7.875 | 5.964
 1.301 | -2.093 | 3.394
 -4.129 | 9.176 | -13.305
 2.827 | -12.433 | 15.261
 -3.757 | 2.013 | -5.771
 -2.726 | 8.997 | -11.722
 -4.818 | -7.253 | 2.434
 -0.915 | -4.439 | 3.524


8.705643341222308